Install Ultralytics

In [ ]:
%pip install ultralytics -qq

In [ ]:
from ultralytics import YOLO

data = "dataset/data.yaml"
epochs = 1000
device = 0
batch = 32
project = "miotcd"
exist_ok = False
resume = True

class YOLOv12:
    def __init__(self, model_path):
        self.model = YOLO(model_path)

    def train(self, data, epochs, device, batch, project, exist_ok,resume):
        return self.model.train(
            data=data,
            epochs=epochs,
            device=device,
            batch=batch,
            project=project,
            exist_ok=exist_ok,
            resume=resume
        )

model_path = "last.pt"
detector = YOLOv12(model_path)

results = detector.train(
    data=data,
    epochs=epochs,
    device=device,
    batch=batch,
    project=project,
    exist_ok=exist_ok,
    resume=resume
)

In [ ]:
from ultralytics import YOLO

class Detector:
    def __init__(self, model_path):
        self.model = YOLO(model_path)

    def detect(self, src_img):
        results = self.model.predict(
            source=src_img,
        )
        bboxes = results[0].boxes.xywh.cpu().numpy()
        bboxes[:,:2] = bboxes[:,:2] - bboxes[:,2:]/2
        scores = results[0].boxes.conf.cpu().numpy()
        class_ids = results[0].boxes.cls.cpu().numpy()
        return bboxes, scores, class_ids
    
model_path = "miotcd/train/weights/best.pt"
src_img = "dataset/images/test/00000035.jpg"
detector = Detector(model_path)
results = detector.detect(src_img)


Validation yolo V12

In [1]:
import sys
import os
sys.path.append("ultralytics")
from ultralytics import YOLO

# Load a model
model = YOLO("best.pt")  # load a custom model
metrics = model.val(data="data.yaml", plots=True)
print(metrics.box.map)  # mAP50-95
print(metrics.box.map50)  # mAP50
print(metrics.box.map75)  # mAP75
print(metrics.box.maps)  # list of mAP50-95 for each category
print(metrics.confusion_matrix.to_json())

# from ultralytics.utils.plotting import plot_results

# # Ví dụ với file results.csv trong train7
# plot_results("miotcd_test/train/results.csv")

Ultralytics 8.3.202 🚀 Python-3.13.3 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 7806MiB)
Model summary (fused): 92 layers, 25,846,129 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 72.2±52.0 MB/s, size: 25.4 KB)
val: Scanning /media/quocphong/01DC253D40D5ABD0/DATA/00_MASTER_COURSE/AAIP536201_TTUDTTNT/Finnal_prj/YOLOv12/dataset/labels/val... 11000 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 11000/11000 4.3Kit/s 2.6s0.1s
val: New cache created: /media/quocphong/01DC253D40D5ABD0/DATA/00_MASTER_COURSE/AAIP536201_TTUDTTNT/Finnal_prj/YOLOv12/dataset/labels/val.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 688/688 5.9it/s 1:56<0.2ss
                   all      11000      35070      0.796      0.796      0.835      0.641
     articulated_truck        799        890      0.846      0.915      0.931      0.799
               bicycle        181        246       0.8

In [ ]:
import cv2
import time
import random
from collections import defaultdict, deque
from ultralytics import YOLO

# ===== Configuration =====
MODEL_PATH = "oversam_best_49ep.pt"      # Path to the trained YOLO model
VIDEO_PATH = "VideoTest/MVI_20011.avi"   # Path to the input video
OUTPUT_PATH = "output_tracking_2.mp4"    # Path to save the output video
TRACKER_YAML = "botsort.yaml"            # Path to the tracker configuration file (e.g., botsort.yaml)

MAX_TRAIL = 30       # Maximum number of points to store for each trail
THICKNESS = 2        # Thickness of the trail line
FONT_SIZE = 0.4      # Font size for annotations (smaller than default)
BOX_THICKNESS = 1    # Bounding box border thickness (thinner)

# ===== Helper Functions =====
def color_for_id(tid: int):
    """Generates a consistent, pseudo-random color for a given track ID."""
    random.seed(tid + 12345)  # Use ID as seed for consistency
    # Generate a bright, non-dark color (B, G, R)
    return (random.randint(40, 255), random.randint(40, 255), random.randint(40, 255))

# ===== Initialization =====
model = YOLO(MODEL_PATH)
cap = cv2.VideoCapture(VIDEO_PATH)

# Get original video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps_video = cap.get(cv2.CAP_PROP_FPS)
# Calculate delay for cv2.waitKey() to match original video speed
delay = int(1000 / fps_video) if fps_video and fps_video > 0 else 1

# Create VideoWriter object to save the result
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Codec for MP4
out = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps_video, (width, height))

# Dictionary to store trails.
# key: track_id, value: deque (a list with a maximum length)
trails = defaultdict(lambda: deque(maxlen=MAX_TRAIL))

print(f"🔍 Processing video {VIDEO_PATH} ...")
print(f"💾 Results will be saved to: {OUTPUT_PATH}")

while True:
    # Read a frame from the video
    ok, frame = cap.read()
    if not ok:
        break  # End of video

    start = time.time() # Start timer for performance measurement

    # Run YOLO tracking on the frame
    results = model.track(
        source=frame,
        tracker=TRACKER_YAML,  # Specify the tracker config
        persist=True,          # Maintain track IDs between frames
        conf=0.25,             # Confidence threshold
        iou=0.45,              # IOU threshold for NMS
        verbose=False,         # Disable detailed console output
        show=False             # Do not display results automatically
    )

    r = results[0]  # Get results for the first (and only) image
    
    # Use the built-in .plot() method to draw boxes and labels
    # We use custom thickness and font size
    annotated = r.plot(line_width=BOX_THICKNESS, font_size=FONT_SIZE)

    boxes = r.boxes
    # Check if any objects were tracked in this frame
    if boxes is not None and boxes.id is not None:
        # Get track IDs and their corresponding bounding boxes
        ids = boxes.id.int().cpu().tolist()
        xyxy = boxes.xyxy.int().cpu().tolist()

        # Update & draw trails
        for (tid, (x1, y1, x2, y2)) in zip(ids, xyxy):
            # Calculate the center point of the bounding box
            cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)
            # Add the new center point to this ID's trail
            trails[tid].append((cx, cy))

        # Draw the trails for all currently tracked objects
        for tid, pts in trails.items():
            if len(pts) > 1: # Need at least two points to draw a line
                col = color_for_id(tid) # Get the color for this ID
                # Draw line segments connecting all points in the trail
                for i in range(1, len(pts)):
                    cv2.line(annotated, pts[i-1], pts[i], col, THICKNESS)

    # Calculate inference time
    end = time.time()
    inf_ms = (end - start) * 1000  # Inference time in milliseconds
    fps = 1 / (end - start + 1e-8) # Frames Per Second

    # Display FPS and inference time on the frame
    text = f"FPS: {fps:.2f} | {inf_ms:.2f} ms"
    cv2.putText(
        annotated, text,
        (annotated.shape[1] - 400, 30), # Position (top-right area)
        cv2.FONT_HERSHEY_SIMPLEX,
        0.6, (0, 255, 0), 2, cv2.LINE_AA
    )

    # Write the annotated frame to the output video
    out.write(annotated)

    # Optional: Display the tracking results in a window
    cv2.imshow("YOLO Tracking (Saving Output)", annotated)
    
    # Exit loop if 'q' is pressed
    if cv2.waitKey(delay) & 0xFF == ord('q'):
        break

# Release all resources
cap.release()
out.release()
cv2.destroyAllWindows()

print("✅ Done! Video saved successfully.")
print(f"📁 File: {OUTPUT_PATH}")